Create local Client

In [11]:
from dask.distributed import Client
client = Client()
client


C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63255 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:63259' processes=4 threads=8, memory=7.89 GiB>

Task exception was never retrieved
future: <Task finished name='Task-2939488' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\client.py", line 2386, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2939494' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\client.py", line 2386, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2939487' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\eleme\.conda\envs\teamx\Lib

In [ ]:
client = Client('127.0.0.1:50211')

In [1]:
import sys
from operator import concat

from xarray import decode_cf

sys.path.append("D:/MSc_Arbeit/model_comparison_codes")
import confg
import xarray as xr
import numpy as np
from functools import partial
import metpy.calc as mpcalc
from metpy.units import units
import datetime
from pathlib import Path
import read_icon_model_3D

In [2]:
lat_ibk = 47.259998
lon_ibk = 11.384167


with h5py.File('ICON2TE_20171015_latlon.nc', 'r') as f:
    for var in f:
        dset = f[var]
        if hasattr(dset, 'chunks') and dset.chunks is not None:
            print(f"{var}: {dset.chunks}")

%%time
icon_10 = xr.open_dataset(confg.icon_folder_3D + "/ICON_20171015_latlon.nc")
# 
# icon_point = icon_point.compute()

In [5]:
def convert_calc_variables(ds):
    """
    Converts and calculates meteorological variables for a xarray Dataset.

    Parameters:
    - df: A xarray Dataset containing the columns 'p' for pressure in Pa
          and 'th' for potential temperature in Kelvin.

    Returns:
    - A xarray Dataset with the original data and new columns:
      'pressure' in hPa and 'temperature' in degrees Celsius.
    """

    # Convert pressure from Pa to hPa
    ds['pressure'] = (ds['pres'] / 100.0) * units.hPa

    # calc pot temp
    ds["th"] = mpcalc.potential_temperature(ds['pressure'], ds["temp"] * units.kelvin)

    # convert temp to °C
    ds["temp"]  = (ds["temp"] - 273.15) * units.degC

    return ds.metpy.dequantify()  # remove units from the dataset

In [3]:
#chunksizes = (1, 90, 712, 1097)
def read_icon_fixed_point(lat, lon, variant="ICON"):
    if variant == "ICON":
        icon_full = xr.open_dataset(confg.icon_folder_3D + "/ICON_20171015_latlon.nc", chunks="auto")  # , chunks={lat': 10, 'lon': 10}

    elif variant == "ICON2TE":
        icon_full = xr.open_dataset(confg.icon2TE_folder_3D + "/ICON2TE_20171015_latlon.nc", chunks={'lat': 10, 'lon': 10})

    icon_point = icon_full.sel(lat=lat, lon=lon, method="nearest")
    icon_point = convert_calc_variables(icon_point)
    icon_point = icon_point.rename({"z_ifc": "z"})
    icon_point = icon_point.compute()
    return icon_point

In [ ]:
%time
read_icon_fixed_point(lat=lat_ibk, lon=lon_ibk, variant="ICON")

CPU times: total: 0 ns
Wall time: 0 ns


In [31]:
%timeit
icon_full = xr.open_dataset(confg.icon_folder_3D + "/ICON_latlon_subset_tirol.nc", chunks={'lat': 2, 'lon': 2})
icon_point = icon_full.sel(lat=lat_ibk, lon=lon_ibk, method="nearest")
icon_point = convert_calc_variables(icon_point)
icon_point = icon_point.rename({"z_ifc": "z"})
icon_point = icon_point.compute()

In [5]:
%%time
icon_point = icon_full.sel(lat=lat_ibk, lon=lon_ibk, method="nearest")

CPU times: total: 0 ns
Wall time: 24.2 ms


In [81]:
%%time
icon_point = icon_point.sel(lat=lat_ibk, lon=lon_ibk, method="nearest")

CPU times: total: 15.6 ms
Wall time: 25.6 ms


In [6]:
%%time
icon_calc = convert_calc_variables(icon_point)

CPU times: total: 31.2 ms
Wall time: 36 ms


In [7]:
%%time
icon_point_100 = icon_calc.compute()  # takes insanely long! >300s!

KeyboardInterrupt: 

## icon fixed time

icon_fixed_time = read_icon_model_3D.read_icon_fixed_time(day=16, hour=12, min=0, variant="ICON")

In [54]:
def read_icon_fixed_time(day=16, hour=12, min=0, variant="ICON"):
    if variant == "ICON":
        icon_full = xr.open_dataset(confg.icon_folder_3D + "/ICON_20171015_latlon.nc",
                                    chunks={'time': 10})
    elif variant == "ICON_2TE":
        icon_full = xr.open_dataset(confg.icon2TE_folder_3D + "/ICON2TE_20171015_latlon.nc",
                                    chunks={'time': 10})
    timestamp = datetime.datetime(2017, 10, day, hour, min, 00)
    icon_point = icon_full.sel(time=timestamp, height=90, method="nearest")
    icon_point = convert_calc_variables(icon_point)
    icon_point = icon_point.compute()
    return icon_point

In [55]:
%%time
icon_extent = read_icon_fixed_time(day=16, hour=12, min=0, variant="ICON2TE")
icon_extent

UnboundLocalError: cannot access local variable 'icon_full' where it is not associated with a value

In [18]:

%%time
import dask.array as da
x = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
y = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
z = (da.arcsin(x) + da.arccos(y)).sum(axis=(1, 2))
z.persist()



CPU times: total: 141 ms
Wall time: 270 ms


dask.array<sum-aggregate, shape=(10000,), dtype=float64, chunksize=(1000,), chunktype=numpy.ndarray>